In [1]:
!pip install psycopg2 tabulate


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import psycopg2

# Database connection configuration
db_config = {
    "dbname": "transaction",
    "user": "postgres",
    "password": "admin",
    "host": "localhost",
    "port": "1234"
}

# Attempt to connect to the database
try:
    conn = psycopg2.connect(**db_config)
    print("✅ Successfully connected to the database!")
    conn.close()  # Close the connection
except Exception as e:
    print("❌ Error:", e)


✅ Successfully connected to the database!


In [3]:
from tabulate import tabulate

def insert_to_db(item, jumlah, harga, total):
    """
    Inserts a transaction record into the PostgreSQL database.

    This function establishes a connection to the database, ensures 
    that the 'transaksi' table exists, and then inserts the given 
    transaction data. If an error occurs, it prints an error message.

    Args:
        item (str): The name of the item.
        jumlah (int): The quantity of the item.
        harga (int): The price per unit.
        total (int): The total price (jumlah * harga).

    Raises:
        Exception: If there is an issue with the database connection or query execution.
    """
    try:
        # Establish a connection to the PostgreSQL database
        conn = psycopg2.connect(**db_config)
        cur = conn.cursor()

        # Ensure the 'transaksi' table exists
        create_table_query = """
        CREATE TABLE IF NOT EXISTS transaksi (
            id SERIAL PRIMARY KEY,
            item TEXT NOT NULL,
            jumlah INT NOT NULL,
            harga INT NOT NULL,
            total_harga INT NOT NULL
        );
        """
        cur.execute(create_table_query)

        # Insert data into the table
        insert_query = """
        INSERT INTO transaksi (item, jumlah, harga, total_harga) 
        VALUES (%s, %s, %s, %s);
        """
        cur.execute(insert_query, (item, jumlah, harga, total))
        conn.commit()  # Commit the transaction
        
        # Close the cursor and connection
        cur.close()
        conn.close()
        print("✅ Data successfully inserted into the database.")
    except Exception as e:
        print("❌ Failed to insert data into the database:", e)


In [4]:
class Belanja:
    def __init__(self):
        """Function for input data customers"""
        self.belanja = []

    def input_belanja(self):
        """Function  for input, 'item', 'jumlah' and 'harga'"""
        
        while True:
            item = input("Masukan Item: ")
            
            try:

                jumlah = int(input("Masukan Jumlah: "))
                
                harga = int(input("Masukan Harga: "))
                
                total = jumlah * harga

                # Insert to database
                insert_to_db(item, jumlah, harga, total)

                self.belanja.append((item, jumlah, harga, total))

                #Except ValueError
            except ValueError:
                print("⚠️ Input harus berupa angka. Silakan coba lagi.")

            lanjut = input("Tambah item lagi? (y/n): ").strip().lower()
            
            #If 'n' is entered, the loop will break and go to the next step
            if lanjut == "n":
                break

    def tampilkan_struk(self):
        """Function to display the shopping receipt in a tabulated format."""
        
        headers = ["Item", "Jumlah", "Harga", "Total"]
        print("\n Struk Belanja:")
        print(tabulate(self.belanja, headers=headers, tablefmt="grid"))

# Eksekusi Program
instance = Belanja()

instance.input_belanja()

instance.tampilkan_struk()


Masukan Item:  Buku
Masukan Jumlah:  2
Masukan Harga:  50000


✅ Data successfully inserted into the database.


Tambah item lagi? (y/n):  n



 Struk Belanja:
+--------+----------+---------+---------+
| Item   |   Jumlah |   Harga |   Total |
+========+==========+=========+=========+
| Buku   |        2 |   50000 |  100000 |
+--------+----------+---------+---------+
